In [1]:
import pandas as pd
import numpy as np
import os
cwd = os.getcwd()
cwd

'/data22/bde/Git/forks/Informer2020'

In [2]:
#https://en.wikipedia.org/wiki/Non-return-to-zero
def NRZ_maker(values):
    NRZ = []
    state = None
    prior = values[0]

    for bit in values:
        if bit == prior:
            state = 1.0
        else:
            state = 0.0
        NRZ.append(state)
    return NRZ

In [3]:
test_array = np.array([6, 6, 6, 12, 12, 12, 6, 12, 12, 12, 6, 12, 12, 12, 6, 12, 12, 12, 6, 12, 12, 12, 6, 12, 12, 12, 6, 12, 12])
test_df = pd.DataFrame({'values': test_array})
values = test_df['values'].tolist()

NRZ = []
state = 0
prior = values[0]
for bit in values:
    if bit == prior:
        state = 1.0
    else:
        state = 0.0
    NRZ.append(state)
#NRZ

test_df['NRZ'] = NRZ
test_df.head()

,values,NRZ
0,6,1.0
1,6,1.0
2,6,1.0
3,12,0.0
4,12,0.0


In [4]:
values = test_df['values'].tolist()
test_df['NRZ2'] = NRZ_maker(values)
test_df.head()

,values,NRZ,NRZ2
0,6,1.0,1.0
1,6,1.0,1.0
2,6,1.0,1.0
3,12,0.0,0.0
4,12,0.0,0.0


In [5]:
PATH = '/data22/bde'
netflix_StrangerThings = f'{PATH}/Data/5Gdataset-master/5G-production-dataset/Netflix/Driving/Season3-StrangerThings/'
netflix_animated_RickandMorty = f'{PATH}/Data/5Gdataset-master/5G-production-dataset/Netflix/Driving/animated-RickandMorty/'
amazon_TheExpanse = f'{PATH}/Data/5Gdataset-master/5G-production-dataset/Amazon_Prime/Driving/Season3-TheExpanse/'
amazon_animated_AdventureTime = f'{PATH}/Data/5Gdataset-master/5G-production-dataset/Amazon_Prime/Driving/animated-AdventureTime/'
download = f'{PATH}/Data/5Gdataset-master/5G-production-dataset/Download/Driving/'
driving_streaming = f'{PATH}/Data/5Gdataset-master/5G-production-dataset/Driving-Streaming/'
driving = f'{PATH}/Data/5Gdataset-master/5G-production-dataset/Driving/'
df_list = []
file_dir = download

for file in os.listdir(file_dir)[:]:
    print(file)
    df = pd.read_csv(file_dir+file)

    values = df['CellID'].tolist()
    df['NRZ'] = NRZ_maker(values)
    df['CellIDChange'] = (df['CellID'] != df['CellID'].shift()).astype('float32')
    
    df = df[['Timestamp', 'CellID', 'NRZ', 'CellIDChange', 'RSRP', 'RSRQ', 'SNR', 'CQI', 'RSSI', 'NRxRSRP', 'NRxRSRQ', 'DL_bitrate', 'UL_bitrate', 'Speed']]
    df = df.replace('-', np.nan)
    df = df.replace('2147483647.0', 0)
    
    df['date'] = pd.to_datetime(df['Timestamp'], format="%Y.%m.%d_%H.%M.%S")
    df = df.drop(columns=['Timestamp'])

    df['CellID'] = pd.to_numeric(df['CellID'], downcast='float')
    df['NRZ'] = pd.to_numeric(df['NRZ'], downcast='float')
    df['RSRP'] = pd.to_numeric(df['RSRP'], downcast='float')
    df['RSRQ'] = pd.to_numeric(df['RSRQ'], downcast='float')
    df['SNR'] = pd.to_numeric(df['SNR'], downcast='float')
    df['CQI'] = pd.to_numeric(df['CQI'], downcast='float')
    df['RSSI'] = pd.to_numeric(df['RSSI'], downcast='float')
    df['NRxRSRP'] = pd.to_numeric(df['NRxRSRP'], downcast='float')
    df['NRxRSRQ'] = pd.to_numeric(df['NRxRSRQ'], downcast='float')
    df['DL_bitrate'] = pd.to_numeric(df['DL_bitrate'], downcast='float')
    df['UL_bitrate'] = pd.to_numeric(df['UL_bitrate'], downcast='float')

    df['RSRPoffset'] = df['RSRP'] - df['NRxRSRP']
    df['RSRQoffset'] = df['RSRQ'] - df['NRxRSRQ']

    #df.fillna(method='ffill', inplace=True)
    df.fillna(0, inplace=True)

    df.drop_duplicates(subset=['date'], inplace=True)
    df.info()
    #df.info(show_counts=True)
    #print(df.head(30))
    #df.plot(subplots=True, layout=(3,3), figsize=(40, 25))
    #df.to_csv('5Gdataset-{}.csv'.format(file), encoding='utf-8', index=False)
    df_list.append(df)

B_2020.02.27_17.30.15.csv
<class 'pandas.core.frame.DataFrame'>
Index: 752 entries, 0 to 865
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   CellID        752 non-null    float32       
 1   NRZ           752 non-null    float32       
 2   CellIDChange  752 non-null    float32       
 3   RSRP          752 non-null    float32       
 4   RSRQ          752 non-null    float32       
 5   SNR           752 non-null    float32       
 6   CQI           752 non-null    float32       
 7   RSSI          752 non-null    float32       
 8   NRxRSRP       752 non-null    float32       
 9   NRxRSRQ       752 non-null    float32       
 10  DL_bitrate    752 non-null    float32       
 11  UL_bitrate    752 non-null    float32       
 12  Speed         752 non-null    int64         
 13  date          752 non-null    datetime64[ns]
 14  RSRPoffset    752 non-null    float32       
 15  RSRQoffset    752 n

In [6]:
df = pd.concat(df_list, axis=0)
df.sort_values('date', inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24446 entries, 0 to 857
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   CellID        24446 non-null  float32       
 1   NRZ           24446 non-null  float32       
 2   CellIDChange  24446 non-null  float32       
 3   RSRP          24446 non-null  float32       
 4   RSRQ          24446 non-null  float32       
 5   SNR           24446 non-null  float32       
 6   CQI           24446 non-null  float32       
 7   RSSI          24446 non-null  float32       
 8   NRxRSRP       24446 non-null  float32       
 9   NRxRSRQ       24446 non-null  float32       
 10  DL_bitrate    24446 non-null  float32       
 11  UL_bitrate    24446 non-null  float32       
 12  Speed         24446 non-null  int64         
 13  date          24446 non-null  datetime64[ns]
 14  RSRPoffset    24446 non-null  float32       
 15  RSRQoffset    24446 non-null  float32      

In [7]:
df.head()

,CellID,NRZ,CellIDChange,RSRP,RSRQ,SNR,CQI,RSSI,NRxRSRP,NRxRSRQ,DL_bitrate,UL_bitrate,Speed,date,RSRPoffset,RSRQoffset
0,12.0,1.0,1.0,-99.0,-15.0,6.0,12.0,-90.0,-100.0,-15.0,7.0,1.0,0,2019-12-14 10:16:30,1.0,0.0
1,12.0,1.0,0.0,-99.0,-15.0,6.0,12.0,-90.0,-100.0,-15.0,7.0,1.0,1,2019-12-14 10:16:31,1.0,0.0
3,12.0,1.0,0.0,-102.0,-14.0,6.0,12.0,-90.0,-99.0,-15.0,0.0,0.0,1,2019-12-14 10:16:32,-3.0,1.0
4,12.0,1.0,0.0,-102.0,-14.0,6.0,12.0,-90.0,-99.0,-15.0,9.0,12.0,1,2019-12-14 10:16:33,-3.0,1.0
5,12.0,1.0,0.0,-102.0,-14.0,4.0,12.0,-90.0,-99.0,-14.0,0.0,0.0,1,2019-12-14 10:16:34,-3.0,0.0


In [8]:
df = df[['date', 'CellID', 'NRZ', 'CellIDChange', 'RSRP', 'RSRQ', 'SNR', 'CQI', 'RSSI', 'NRxRSRP', 'NRxRSRQ', 'RSRPoffset', 'RSRQoffset', 'UL_bitrate', 'DL_bitrate']]

In [9]:
df.to_csv('data/5Gdataset/download.csv', encoding='utf-8', index=False)

In [10]:
# number of changes in values
(df != df.shift(axis=0)).sum(axis=0)

date            24446
CellID            699
NRZ               296
CellIDChange     1280
RSRP             9295
RSRQ             7963
SNR              9224
CQI              7725
RSSI             8710
NRxRSRP          7488
NRxRSRQ          7562
RSRPoffset       9045
RSRQoffset       8102
UL_bitrate      22262
DL_bitrate      22631
dtype: int64